In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import json
import openai
import random
import os
import copy
import collections 

with open('params.json') as f:
    params = json.load(f)

openai.api_key = params['OPENAI_API_KEY']
openai.organization = params['OPENAI_ORG']

def get_response(prompt, model='gpt-3.5-turbo', temperature=0.9):
    result = openai.ChatCompletion.create(
    model=model,
    temperature=temperature,
    messages=[
            {"role": "system", "content": "You are mimicking a real-life person who wants to make friends."},
            {"role": "user", "content": prompt},
    ])

    return result.choices[0]['message']['content']


def draw_graph(G, ax, G0=None):
    pos = nx.spring_layout(G)

    if not G0:
        nx.draw(G, pos, ax=ax, node_size=10, width=0.1, node_color='green', alpha=0.7, edge_color='black')
    else:
        G0_edges = set(G0.edges())
        G_edges = set(G.edges()) - G0_edges

        nx.draw_networkx_edges(G, pos, edgelist=G0_edges, width=0.1, alpha=0.5, edge_color='black', ax=ax)
        nx.draw_networkx_edges(G, pos, edgelist=G_edges, width=0.5, alpha=1, edge_color='red', ax=ax)

        nx.draw_networkx_nodes(G, pos, nodelist=list(G.nodes()), node_size=10, node_color='green', alpha=0.7, ax=ax)

        ax.set_axis_off()

def create_preferrential_attachment(T, n0, temperature):
    G = nx.erdos_renyi_graph(n0, 0.5)

    Gs = []
    results = []

    for t in range(n0, n0 + T):
        print(f'Adding node {t}')
        if t > 0:
            result = select_neighbor(G, t, temperature)
        G.add_node(t)
        
        if t > 0 and result:
            v = result['name']
            G.add_edge(t, v)

        Gs.append(G.copy())
        results.append(result)

    return Gs, results

def select_neighbor(G, t, temperature):
    degrees = []
    for v in G.nodes():
        if v != t:
            degrees.append({'name' : v, 'number_of_friends' : G.degree(v)})

    preferential_attachment_prompt = f"""
    # Task
    Your task is to select a person to be friends with.

    # Input
    The input is a list of dictionaries. Each dictionary has two keys: 'name' and 'number of friends'.
    'name' is the name of the person and 'number_of_friends' are the number of friends of the person.
    The data is given below after chevrons:
    <DEGREES>
    {json.dumps(degrees, indent=4)}
    </DEGREES>

    # Output
    The output should be given in JSON format with the following structure

    {{
        "name" : name of the person you selected,
        "number_of_friends" : the number of friends of the person you selected,
        "reason" : reason for selecting the person
    }}


    ```json
    """

    for i in range(10):
        try:
            result = json.loads(get_response(preferential_attachment_prompt, temperature=temperature).split('```')[0])
            if result['name'] in G.nodes():
                print('NODE DEGREES', degrees)
                print('NEW EDGE', result)
                return result
        except Exception as e:
            print(e)
        
def run_network_formation_experiment(n_min, n_max, n_step, num_simulations, outfile, temperatures):

    saved_scenarios = set()

    if os.path.exists(outfile):
        with open(outfile) as f:
            lines = f.read().splitlines()

            for line in lines:
                scenario = json.loads(line)
                saved_scenarios.add((scenario['n'], scenario['simulation'], scenario['temperature']))

        exit()

    f = open(outfile, 'a+')

    print(saved_scenarios)

    for n in range(n_min, n_max + 1, n_step):
        for i in range(num_simulations):
            for temperature in temperatures:
                if (n, i, temperature) in saved_scenarios:
                    print(f'Skipping simulation for n={n}, i={i}, temperature={temperature}')
                    continue
                else:
                    print(f'Running simulation for n={n}, i={i}, temperature={temperature}')
                    n0 = int(np.ceil(np.sqrt(n)))
                    Gs, reasons = create_preferrential_attachment(n, n0, temperature=temperature)

                    temp = {
                        'n' : n,
                        'n0' : n0,
                        'temperature' : temperature,
                        'simulation' : i,
                        'graphs' : [nx.to_dict_of_lists(G) for G in Gs],
                        'reasons' : reasons
                    }    

                    f.write(json.dumps(temp) + '\n')            

    f.close()

def analyze_experiments(filename):

    with open(filename) as f:
        lines = f.read().splitlines()

    data = []

    for line in lines:
        data.append(json.loads(line))

    degree_freqs = collections.defaultdict(list)
    dergee_freqs_barabasi_albert = collections.defaultdict(list)

    for d in data:
        Gs = []
        for graph in d['graphs']:
            G = nx.Graph()

            for k, v in graph.items():
                k = int(k)
                G.add_node(k)
                for n in v:
                    G.add_edge(k, n)
            G.remove_nodes_from(list(nx.isolates(G)))
            Gs.append(G)

        fig, ax = plt.subplots(1, 4, figsize=(20, 5))

        fig.suptitle(f'Graph created based on Principle 1 with $n = {d["n"]}$, $n_0={d["n0"]}$, temperature={d["temperature"]}')

        for i, t in enumerate([0, len(Gs) // 2, len(Gs) - 1]):
            G = Gs[t]
            ax[i].set_title(f'$t = {t}$')
            draw_graph(G, ax=ax[i], G0=Gs[0])

        degree_freq = nx.degree_histogram(G)
        degree_freqs[d['n'], d['temperature']].append(degree_freq)

    
        G_barabasi_albert = nx.barabasi_albert_graph(n=d['n'], m=1, initial_graph=Gs[0], seed=1)

        degree_freq_barabasi_albert = nx.degree_histogram(G_barabasi_albert)
        dergee_freqs_barabasi_albert[d['n'], d['temperature']].append(degree_freq_barabasi_albert)

        degrees = range(max(len(degree_freq), len(degree_freq_barabasi_albert)))

        ax[-1].set_title('Degree distribution')
        ax[-1].loglog(degree_freq,'go-', label='LLM')
        ax[-1].loglog(degree_freq_barabasi_albert,'bo-', label='BA') 
        ax[-1].legend()
        ax[-1].set_xlabel('Degree')
        ax[-1].set_ylabel('Frequency')

        fig.tight_layout()

        fig.savefig(f'figures/principle_1/principle_1_{d["n"]}_{d["simulation"]}_{d["temperature"]}.png')

    fig, ax = plt.subplots(1, len(degree_freqs), figsize=(5 * len(degree_freqs), 5))

    for i, k in enumerate(sorted(degree_freqs)):

        degree_freq = np.mean(degree_freqs[k], axis=0)
        degree_freq_barabasi_albert = np.mean(dergee_freqs_barabasi_albert[k], axis=0)

        ax[i].set_title(f'$n = {k[0]}$, temperature={k[1]}')
        ax[i].loglog(degree_freq,'go-', label='LLM')
        ax[i].loglog(degree_freq_barabasi_albert,'bo-', label='BA') 
        ax[i].set_xlabel('Degree')
        ax[i].set_ylabel('Frequency')

    fig.tight_layout()
    fig.savefig(f'figures/principle_1/principle_1_overall.png')

In [ ]:
run_network_formation_experiment(50, 50, 1, 10, 'outputs/principle_1_gpt-3.5-turbo.jsonl', [0.5, 1.0, 1.5])
analyze_experiments('outputs/principle_1_gpt-3.5-turbo.jsonl')